En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

PREPARACION:
1.   Commit inicial del proyecto (hecho con GitHub desktop)
3.   Instalar Git Flow con brew install git-flow
4.   Configurar repositorio con git flow init
5.   Configurar que los feature finish se hagan solo en develop con git config gitflow.feature.finish.keepremote true
6.   Sincronizar repositorio con GDrive
7.   Leer código desde GDrive con Colab

In [2]:
# Project and dataset information
SOURCE_PATH = '/content/drive/Othercomputers/My Mac/latam-challenge'
PROJECT_ID = "tw-techdash"
DATASET_NAME = "tweets_dataset"
TABLE_NAME = "tweets"
BUCKET_NAME = 'tw-gcp-public-lab'
ZIP_FILE = "tweets.json.zip"
FILE_ID = '1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis'
SOURCE_URI = f"gs://{BUCKET_NAME}/raw/"

In [36]:
# Automatically reload the Jupyter kernel when it detects changes in the project directory
%reload_ext autoreload

# IPython will attempt to automatically reload any module you've previously imported before executing a cell
%autoreload 2

In [37]:
from google.colab import drive

# Mount Google Drive to the path /content/drive
# Force remounting in case of errors or issues
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [38]:
# Import the necessary libraries in the virtual environment according to the requirements.txt file
# The purpose of this is to make the notebook work in any environment
import sys
!{sys.executable} -m pip install -r '{SOURCE_PATH}/requirements.txt'

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/pip/__main__.py", line 9, in <module>
    if sys.path[0] in ("", os.getcwd()):
OSError: [Errno 107] Transport endpoint is not connected


In [39]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [ ]:
%cd {SOURCE_PATH}/src

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-40-ef3161532441>", line 1, in <cell line: 1>
    get_ipython().run_line_magic('cd', '{SOURCE_PATH}/src')
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactivesh

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-40-ef3161532441>", line 1, in <cell line: 1>
    get_ipython().run_line_magic('cd', '{SOURCE_PATH}/src')
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactivesh

In [40]:
import q1_time

# Llama a la función q1_time para obtener el resultado
resultado = q1_time.q1_time(file_path)

# Imprime el resultado
print(resultado)

Ejecutando q1_time con file_path: farmers-protest-tweets-2021-2-4.json
[(datetime.date(2022, 1, 1), 'Registro 1'), (datetime.date(2022, 1, 2), 'Registro 2')]


In [4]:
from google.colab import auth
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload
from google.cloud import storage
import zipfile

def authenticate_google_drive():
    """Authenticates to Google Drive if needed."""
    auth.authenticate_user()

def download_file_from_drive(file_id):
    """Downloads a file from Google Drive."""
    drive_service = build('drive', 'v3')
    request = drive_service.files().get_media(fileId=file_id)
    downloaded = io.BytesIO()
    downloader = MediaIoBaseDownload(downloaded, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f'Downloading {int(status.progress() * 100)}%')
    downloaded.seek(0)
    return downloaded

def upload_file_to_cloud_storage(downloaded, bucket_name, folder_name):
    """Uploads a file to Google Cloud Storage."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)

    # Check and create folder
    if not bucket.blob(f"{folder_name}/"):
        bucket.blob(f"{folder_name}/").upload_from_string('', content_type='application/x-www-form-urlencoded;charset=UTF-8')

    blob = bucket.blob(f'{folder_name}/{ZIP_FILE}')
    blob.upload_from_file(downloaded, content_type='application/zip')
    print(f'Archivo subido a gs://{bucket_name}/{blob.name}')

    return client, bucket, blob

def decompress_zip_file(downloaded, bucket, folder_name):
    """Decompresses a ZIP file in Google Cloud Storage."""
    try:
        with zipfile.ZipFile(downloaded, 'r') as z:
            for file_info in z.infolist():
                with z.open(file_info) as file:
                    blob_name = f'{folder_name}/{file_info.filename}'
                    blob = bucket.blob(blob_name)
                    blob.upload_from_file(file)
        print(f'Archivo descomprimido en gs://{bucket}/{blob_name}')
    except zipfile.BadZipFile:
        print(f'El archivo en gs://{bucket}/{folder_name}/ no es un archivo ZIP válido.')
    finally:
      return file_info.filename

try:
    authenticate_google_drive()
    downloaded = download_file_from_drive(FILE_ID)
    client, bucket, blob = upload_file_to_cloud_storage(downloaded, 'tw-gcp-public-lab', 'raw')
    file_name = decompress_zip_file(downloaded, bucket, 'raw')
except Exception as e:
    print(f'Error: {e}')
finally:
    downloaded.close()  # Close downloaded file


Archivo subido a gs://tw-gcp-public-lab/raw/tweets.json.zip
Archivo descomprimido en gs://<Bucket: tw-gcp-public-lab>/raw/farmers-protest-tweets-2021-2-4.json


In [75]:
# TODO: ANALIZAR BORRAR ESTE BLOQUE
from google.cloud import bigquery
from google.api_core.exceptions import NotFound

def authenticate_bigquery():
    """Authenticates to BigQuery."""
    return bigquery.Client(project=PROJECT_ID)

def create_dataset_if_not_exists(client, dataset_name):
    """Creates a dataset if it does not exist."""
    dataset_ref = client.dataset(dataset_name)
    try:
        client.get_dataset(dataset_ref)
        print(f"Dataset '{dataset_name}' already exists.")
    except NotFound:
        client.create_dataset(dataset_ref)
        print(f"Dataset '{dataset_name}' created.")

def create_table_if_not_exists(client, dataset_name, table_name, schema):
    """Creates a table if it does not exist."""
    dataset_ref = client.dataset(dataset_name)
    table_ref = dataset_ref.table(table_name)
    try:
        client.get_table(table_ref)
        print(f"Table '{table_name}' already exists.")
    except NotFound:
        table = bigquery.Table(table_ref, schema=schema)
        client.create_table(table)
        print(f"Table '{table_name}' created.")

# Authenticate to BigQuery
client = authenticate_bigquery()

# Define table schema
SCHEMA = [
    bigquery.SchemaField('created_at', 'STRING'),
    bigquery.SchemaField('username', 'STRING'),
    bigquery.SchemaField('text', 'STRING')
]

# Create dataset if it does not exist
create_dataset_if_not_exists(client, DATASET_NAME)

# Create table if it does not exist
create_table_if_not_exists(client, DATASET_NAME, TABLE_NAME, SCHEMA)

# Load data from Cloud Storage to BigQuery
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

load_job = client.load_table_from_uri(
    SOURCE_URI+file_name,
    client.dataset(DATASET_NAME).table(TABLE_NAME),
    job_config=job_config
)
#load_job.result()  # Wait for the job to finish
print(f"Data loaded from '{SOURCE_URI}' to table '{TABLE_NAME}'.")


Dataset 'tweets_dataset' already exists.
Table 'tweets' already exists.
Data loaded from 'gs://tw-gcp-public-lab/raw/' to table 'tweets'.


In [43]:
# TODO: ANALIZAR BORRAR ESTE BLOQUE
from google.cloud import storage
import json

# Autenticar con Google Cloud Storage
storage_client = storage.Client()

# Especificar la ubicación del archivo en GCS
bucket_name = "tw-gcp-public-lab"
file_name = "raw/farmers-protest-tweets-2021-2-4.json"

# Obtener el archivo de GCS
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_name)

# Leer el contenido del archivo JSON línea por línea
data = blob.download_as_string().decode('utf-8')



In [76]:
from google.cloud import bigquery
from google.api_core.exceptions import NotFound

def authenticate_bigquery():
    """Authenticates to BigQuery."""
    return bigquery.Client(project=PROJECT_ID)

def create_dataset_if_not_exists(client, dataset_name):
    """Creates a dataset if it does not exist."""
    dataset_ref = client.dataset(dataset_name)
    try:
        client.get_dataset(dataset_ref)
        print(f"Dataset '{dataset_name}' already exists.")
    except NotFound:
        client.create_dataset(dataset_ref)
        print(f"Dataset '{dataset_name}' created.")

def create_table_if_not_exists(client, dataset_name, table_name):
    """Creates a table if it does not exist."""
    dataset_ref = client.dataset(dataset_name)
    table_ref = dataset_ref.table(table_name)
    try:
        client.get_table(table_ref)
        print(f"Table '{table_name}' already exists.")
    except NotFound:
        table = bigquery.Table(table_ref)
        table.schema = []  # Empty schema indicates that BigQuery should infer the schema
        client.create_table(table)
        print(f"Table '{table_name}' created.")

# Authenticate to BigQuery
client = authenticate_bigquery()

# Create dataset if it does not exist
create_dataset_if_not_exists(client, DATASET_NAME)

# Create table if it does not exist
create_table_if_not_exists(client, DATASET_NAME, TABLE_NAME)

# Load data from Cloud Storage to BigQuery
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
job_config.autodetect = True  # Allow BigQuery to autodetect the schema
job_config.ignore_unknown_values = True  # Ignore unknown values in JSON
load_job = client.load_table_from_uri(
    SOURCE_URI+file_name,
    client.dataset(DATASET_NAME).table(TABLE_NAME),
    job_config=job_config
)
load_job.result()


Dataset 'tweets_dataset' already exists.
Table 'tweets' already exists.


LoadJob<project=tw-techdash, location=US, id=469ad64f-ee7c-4861-a1d2-62ea4417bc1e>

In [78]:
query = """
select user.profileBannerUrl
FROM {dataset}.{table}
limit 3
""".format(dataset=DATASET_NAME, table=TABLE_NAME)


query_job = client.query(query)
results = query_job.result()
#for row in results:
    #print(row)
for row in results:
    print(row)
    profile_banner_url = row[0]  # Acceder al primer elemento de la tupla
    print(profile_banner_url)

Row(('https://pbs.twimg.com/profile_banners/873606662552268801/1595003990',), {'profileBannerUrl': 0})
https://pbs.twimg.com/profile_banners/873606662552268801/1595003990
Row(('https://pbs.twimg.com/profile_banners/1090885330163494914/1612202802',), {'profileBannerUrl': 0})
https://pbs.twimg.com/profile_banners/1090885330163494914/1612202802
Row(('https://pbs.twimg.com/profile_banners/819333425203150848/1614029794',), {'profileBannerUrl': 0})
https://pbs.twimg.com/profile_banners/819333425203150848/1614029794
